#Predicción de Rendimiento con XGBoost.

Este notebook implementa un clasificador XGBoost para predecir el rendimiento académico de estudiantes basándose en datos de las pruebas Saber Pro de Colombia.

## Objetivo:
Predecir el `RENDIMIENTO_GLOBAL` de estudiantes en 4 categorías:
- Alto
- Medio-Alto
- Medio-Bajo
- Bajo

---

## Paso 1: Cargar credenciales de Kaggle

Primero, subimos el archivo `kaggle.json` que contiene las credenciales de la API de Kaggle para poder descargar los datos de la competencia.

In [ ]:
from google.colab import files
files.upload()

##Descargar datos de Kaggle:

Configuramos las credenciales de Kaggle y descargamos los datasets de la competencia:
- `train. csv`: Datos de entrenamiento con la variable objetivo
- `test.csv`: Datos de prueba sin la variable objetivo
- `submission_example.csv`: Ejemplo del formato de envío

El comando `wc *.csv` muestra el número de líneas, palabras y bytes de cada archivo.

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '. '

!chmod 600 kaggle.json
! kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia
! unzip *.zip > /dev/null
!wc *.csv

##Importar librerías necesarias:

Importamos las bibliotecas que utilizaremos:
- **pandas**: Manipulación de datos
- **numpy**: Operaciones numéricas
- **sklearn**: Herramientas de machine learning (división de datos, encoding, métricas)
- **xgboost**: Algoritmo de gradient boosting para clasificación

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import xgboost as xgb

##Paso 4: Definir la clase clasificadora

*   Elemento de la lista
*   Elemento de la lista



### `StudentPerformanceXGBoostClassifier`

Esta clase encapsula todo el pipeline de machine learning:

###**Métodos de limpieza:**
- `clean_periodo()`: Separa PERIODO_ACADEMICO en AÑO y PERIODO
- `clean_internet()`: Consolida columnas duplicadas de internet
- `clean_estrato()`: Normaliza valores de estrato socioeconómico ("Estrato 3" → 3)
- `clean_valormatricula()`: Convierte rangos de matrícula a valores ordinales (1-7)
- `clean_horastrabajo()`: Convierte rangos de horas trabajadas a valores ordinales (0-4)
- `clean_educacion()`: Mapea niveles educativos a escala ordinal (0-9)
- `clean_binary()`: Convierte Si/No a 1/0

### **Feature Engineering:**
- `feature_engineering()`: Crea nuevas características:
  - Variables binarias para bienes del hogar
  - Promedio, máximo, mínimo y diferencia de educación de padres
  - Score de bienes del hogar (ponderado)
  - Score socioeconómico integrado
  - Estadísticos de los indicadores (promedio, max, min)

###**Encoding:**
- `encode_categorical()`: Codifica variables categóricas sin orden lógico usando LabelEncoder

### **Pipeline:**
- `preprocess_data()`: Ejecuta todo el preprocesamiento en orden
- `train_model()`: Entrena el modelo XGBoost con hiperparámetros optimizados
- `evaluate()`: Evalúa el modelo y muestra métricas
- `save_predictions()`: Genera archivo de submission para Kaggle

In [ ]:
class StudentPerformanceXGBoostClassifier:
    def __init__(self):
        self.label_encoders = {}
        self.model = None
        self.target_column = 'RENDIMIENTO_GLOBAL'

    #limpieza
    def clean_periodo(self, df):
        df['AÑO'] = df['PERIODO_ACADEMICO'].astype(str). str[:4]. astype(int)
        df['PERIODO'] = df['PERIODO_ACADEMICO'].astype(str).str[4:].astype(int)
        df. drop(columns=['PERIODO_ACADEMICO'], inplace=True)
        return df

    def clean_internet(self, df):
        if 'F_TIENEINTERNET. 1' in df.columns:
            df['F_TIENEINTERNET'] = df['F_TIENEINTERNET.1'].fillna(df['F_TIENEINTERNET'])
            df.drop(columns=['F_TIENEINTERNET.1'], inplace=True)
        return df

    def clean_estrato(self, x):
        if pd.isna(x):
            return 0
        x = str(x).lower(). strip()
        x = x.replace("estrato", "").strip()
        return int(x) if x.isdigit() else 0

    def clean_valormatricula(self, x):
        if pd.isna(x):
            return 0

        matricula_map = {
            'menos de 500 mil': 1,
            'entre 500 mil y menos de 1 millón': 2,
            'entre 1 millón y menos de 2. 5 millones': 3,
            'entre 2.5 millones y menos de 4 millones': 4,
            'entre 4 millones y menos de 5. 5 millones': 5,
            'entre 5.5 millones y menos de 7 millones': 6,
            'más de 7 millones': 7,
            'no pagó matrícula': 0
        }

        x_lower = str(x).lower(). strip()
        return matricula_map.get(x_lower, 0)

    def clean_horastrabajo(self, x):
        if pd.isna(x):
            return 0

        x_str = str(x).lower().strip()

        if x_str == '0':
            return 0
        elif 'menos de 10' in x_str:
            return 1
        elif 'entre 11 y 20' in x_str:
            return 2
        elif 'entre 21 y 30' in x_str:
            return 3
        elif 'más de 30' in x_str or 'mas de 30' in x_str:
            return 4
        else:
            return 0

    def clean_educacion(self, x):
        if pd.isna(x):
            return 2

        edu_map = {
            'ninguno': 0,
            'primaria incompleta': 1,
            'primaria completa': 2,
            'secundaria (bachillerato) incompleta': 3,
            'secundaria (bachillerato) completa': 4,
            'técnica o tecnológica incompleta': 5,
            'técnica o tecnológica completa': 6,
            'educación profesional incompleta': 7,
            'educación profesional completa': 8,
            'postgrado': 9,
            'no sabe': 2,
            'no aplica': 2
        }

        x_lower = str(x).lower().strip()
        return edu_map.get(x_lower, 2)

    def clean_binary(self, x):
        if pd.isna(x):
            return 0
        x_str = str(x).lower(). strip()
        return 1 if x_str in ['si', 'sí', 's', 'y'] else 0


    #feature engineering
    def feature_engineering(self, df):

        # Limpiar columna duplicada de internet PRIMERO
        df = self.clean_internet(df)

        # convertir variables binarias
        df['INTERNET_BIN'] = df['F_TIENEINTERNET'].apply(self.clean_binary)
        df['LAVADORA_BIN'] = df['F_TIENELAVADORA'].apply(self. clean_binary)
        df['AUTOMOVIL_BIN'] = df['F_TIENEAUTOMOVIL'].apply(self.clean_binary)
        df['COMPUTADOR_BIN'] = df['F_TIENECOMPUTADOR'].apply(self.clean_binary)
        df['PAGOMATRICULA_BIN'] = df['E_PAGOMATRICULAPROPIO'].apply(self.clean_binary)
        df['PRIVADO_LIBERTAD_BIN'] = df['E_PRIVADO_LIBERTAD'].apply(lambda x: 1 if str(x).upper() == 'S' else 0)

        # Convertir variables numéricas ordinales
        df['F_ESTRATOVIVIENDA_NUM'] = df['F_ESTRATOVIVIENDA'].apply(self.clean_estrato)
        df['E_VALORMATRICULAUNIVERSIDAD_NUM'] = df['E_VALORMATRICULAUNIVERSIDAD'].apply(self.clean_valormatricula)
        df['E_HORASSEMANATRABAJA_NUM'] = df['E_HORASSEMANATRABAJA']. apply(self.clean_horastrabajo)
        df['F_EDUCACIONPADRE_NUM'] = df['F_EDUCACIONPADRE'].apply(self.clean_educacion)
        df['F_EDUCACIONMADRE_NUM'] = df['F_EDUCACIONMADRE'].apply(self.clean_educacion)

        # features derivadas de educación
        df['FAMI_EDUCACION_PROMEDIO'] = (df['F_EDUCACIONPADRE_NUM'] + df['F_EDUCACIONMADRE_NUM']) / 2
        df['FAMI_EDUCACION_MAX'] = df[['F_EDUCACIONPADRE_NUM', 'F_EDUCACIONMADRE_NUM']].max(axis=1)
        df['FAMI_EDUCACION_MIN'] = df[['F_EDUCACIONPADRE_NUM', 'F_EDUCACIONMADRE_NUM']].min(axis=1)
        df['FAMI_EDUCACION_DIFERENCIA'] = abs(df['F_EDUCACIONPADRE_NUM'] - df['F_EDUCACIONMADRE_NUM'])

        # Score de bienes del hogar
        df['SCORE_BIENES'] = (
            df['LAVADORA_BIN'] * 0.2 +
            df['AUTOMOVIL_BIN'] * 0.4 +
            df['COMPUTADOR_BIN'] * 0.3 +
            df['INTERNET_BIN'] * 0. 1
        )

        # Score socioeconómico integrado
        df['SCORE_SOCIOECONOMICO'] = (
            df['F_ESTRATOVIVIENDA_NUM'] * 0.3 +
            df['E_VALORMATRICULAUNIVERSIDAD_NUM'] * 0.3 +
            df['SCORE_BIENES'] * 0.4
        )

        # Usar indicadores si existen
        if 'INDICADOR_1' in df.columns:
            df['INDICADOR_PROMEDIO'] = df[['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']].mean(axis=1)
            df['INDICADOR_MAX'] = df[['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']]. max(axis=1)
            df['INDICADOR_MIN'] = df[['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']].min(axis=1)

        return df

    #Encoding
    def encode_categorical(self, df, is_training=True):

        categorical_columns = ['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO']

        for col in categorical_columns:
            if col in df. columns:
                if is_training:
                    le = LabelEncoder()
                    df[col] = le.fit_transform(df[col]. astype(str))
                    self.label_encoders[col] = le
                else:
                    le = self.label_encoders[col]
                    df[col] = df[col].astype(str). apply(lambda x: x if x in le.classes_ else le.classes_[0])
                    df[col] = le.transform(df[col])

        return df

    #preprocesado
    def preprocess_data(self, df, is_training=True):

        df = df.copy()
        ids = df['ID']. copy()

        df = self.clean_periodo(df)
        df = self.feature_engineering(df)
        df = self.encode_categorical(df, is_training=is_training)

        #eliminar columnas originales ya procesadas
        columns_to_drop = [
            'F_TIENEINTERNET', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL',
            'F_TIENECOMPUTADOR', 'E_PAGOMATRICULAPROPIO', 'E_PRIVADO_LIBERTAD',
            'F_ESTRATOVIVIENDA', 'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA',
            'F_EDUCACIONPADRE', 'F_EDUCACIONMADRE'
        ]

        df. drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

        #Verificar que no queden columnas object
        object_cols = df.select_dtypes(include=['object']).columns. tolist()
        if object_cols and self.target_column in object_cols:
            object_cols.remove(self.target_column)

        if object_cols:
            df. drop(columns=object_cols, inplace=True, errors='ignore')

        #separar target y features
        if is_training and self.target_column in df.columns:
            y = df[self.target_column]
            X = df.drop(columns=[self.target_column, 'ID'])
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
            return X_train, X_test, y_train, y_test
        else:
            df.drop(columns=['ID'], inplace=True, errors='ignore')
            return df, ids

    #Entrenamiento parametros
    def train_model(self, X_train, y_train):

        if y_train.dtype == 'object':
            self.target_encoder = LabelEncoder()
            y_train_encoded = self.target_encoder. fit_transform(y_train)
        else:
            y_train_encoded = y_train

        self.model = xgb.XGBClassifier(
            n_estimators=450,
            learning_rate=0.04,
            max_depth=9,
            subsample=0.8,
            colsample_bytree=0.8,
            objective='multi:softmax',
            num_class=len(np.unique(y_train_encoded)),
            tree_method='hist',
            random_state=42
        )

        self.model.fit(X_train, y_train_encoded)
        print("-->>Entrenamiento completado<<--")

    #Evaluacion y reportes
    def evaluate(self, X_test, y_test):

        if y_test.dtype == 'object':
            y_test_encoded = self.target_encoder.transform(y_test)
        else:
            y_test_encoded = y_test

        preds_encoded = self.model.predict(X_test)

        if hasattr(self, 'target_encoder'):
            preds = self.target_encoder. inverse_transform(preds_encoded)
            y_test_original = y_test
        else:
            preds = preds_encoded
            y_test_original = y_test

        print("Reporte de Clasificación:")
        print(classification_report(y_test_original, preds))
        print("Matriz de Confusión:")
        print(confusion_matrix(y_test_original, preds))

        return preds

    #Guardar archivo
    def save_predictions(self, df_test, filename='submission. csv'):

        df_test_processed, ids = self.preprocess_data(df_test, is_training=False)
        preds_encoded = self.model.predict(df_test_processed)

        if hasattr(self, 'target_encoder'):
            preds = self.target_encoder.inverse_transform(preds_encoded)
        else:
            preds = preds_encoded

        submission = pd.DataFrame({"ID": ids, "RENDIMIENTO_GLOBAL": preds})
        submission. to_csv(filename, index=False)
        print(f"Archivo guardado como: {filename}")

        return submission

##Cargar los datos

Cargamos los archivos CSV descargados de Kaggle:
- **train.csv**: Dataset de entrenamiento (~692,500 registros)
- **test.csv**: Dataset de prueba (~296,786 registros)

In [ ]:
# Cargar datos
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

##Inicializar el clasificador

Creamos una instancia de la clase `StudentPerformanceXGBoostClassifier` que contiene todo el pipeline de ML.

In [ ]:
clf = StudentPerformanceXGBoostClassifier()

##Preprocesar datos y entrenar modelo

### Preprocesamiento:
1.  Limpia y normaliza todas las columnas
2. Crea features derivadas (scores socioeconómicos, promedios de educación, etc.)
3. Codifica variables categóricas
4. Divide datos en train (80%) y test (20%) con estratificación

### Entrenamiento:
Entrena un modelo XGBoost con los siguientes hiperparámetros:
- **n_estimators**: 450 árboles
- **learning_rate**: 0.04
- **max_depth**: 9 niveles por árbol
- **subsample**: 0. 8 (usa 80% de datos por árbol)
- **colsample_bytree**: 0.8 (usa 80% de features por árbol)

In [ ]:
# Preprocesar y entrenar
X_train, X_test, y_train, y_test = clf.preprocess_data(df_train, is_training=True)
clf.train_model(X_train, y_train)

##Evaluar el modelo

Evaluamos el rendimiento del modelo en el conjunto de validación (20% de train).

### Métricas mostradas:
- **Precision**: De las predicciones de cada clase, cuántas fueron correctas
- **Recall**: De todos los casos reales de cada clase, cuántos se detectaron
- **F1-score**: Media armónica entre precision y recall
- **Support**: Número de casos reales de cada clase
- **Matriz de confusión**: Detalle de predicciones correctas e incorrectas por clase

**Interpretación:** Un accuracy del ~43% indica que el modelo tiene dificultad para distinguir entre las 4 clases, especialmente entre las categorías intermedias (medio-alto y medio-bajo).

In [ ]:
# Evaluar
clf.evaluate(X_test, y_test)

##Generar predicciones para Kaggle

Aplicamos el modelo entrenado sobre el dataset de test y generamos el archivo de submission.

### Proceso:
1.  Preprocesa el dataset de test usando el mismo pipeline que train
2.  Genera predicciones para cada estudiante
3. Crea un DataFrame con formato Kaggle:
   - **ID**: Identificador del estudiante
   - **RENDIMIENTO_GLOBAL**: Predicción (alto, medio-alto, medio-bajo, bajo)
4. Guarda como `my_submission.csv`

###Resultado:
Archivo listo para subir a la competencia de Kaggle con 296,786 predicciones.

In [ ]:
# Predecir en test
clf.save_predictions(df_test, 'my_submission.csv')